Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import os, re
from getpass import getpass

with open('demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [2]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.6/341.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.0/207.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.5/236.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.5/517.5 kB 31.5 MB/s eta 0:00:00
   ━

Import packages

In [3]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, get_available_publicips, get_segment_ids, get_worker_interfaces

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network.

In [4]:
# ep5g reservation
ep5g_lease = show_reservation_byname("ep5g-lease")
segment_ids = get_segment_ids('ep5g')
if not ep5g_lease:
    ep5g_lease = reserve(
        { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2024-11-29 12:28:44.904 | INFO     | chi.expeca:reserve:243 - reserving ep5g
2024-11-29 12:28:47.535 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for ep5g-lease with id 7908b6dc-d2be-435b-acb4-5b400ec1f13e to become "ACTIVE"
2024-11-29 12:28:52.735 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 7908b6dc-d2be-435b-acb4-5b400ec1f13e is PENDING.
2024-11-29 12:28:57.925 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 7908b6dc-d2be-435b-acb4-5b400ec1f13e is PENDING.
2024-11-29 12:29:03.110 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 7908b6dc-d2be-435b-acb4-5b400ec1f13e is STARTING.
2024-11-29 12:29:08.311 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 7908b6dc-d2be-435b-acb4-5b400ec1f13e is STARTING.
2024-11-29 12:29:13.510 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 7908b6dc-d2be-435b-acb4-5b400ec1f

[
    {
        "name": "ep5g-lease",
        "id": "7908b6dc-d2be-435b-acb4-5b400ec1f13e",
        "reservation_id": "3defb345-73e9-4b43-8bc6-6c9d2d5d0de3",
        "status": "ACTIVE",
        "end_date": "2024-12-06T12:28:00.000000"
    }
]


In the following section we setup the networking equipment for ep5g and Internet (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info).

In [5]:
# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router", "public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the ep5g route added to the router")

2024-11-29 12:29:32.706 | SUCCESS  | __main__:<cell line: 3>:3 - ep5g-vip-router router is created.
2024-11-29 12:29:32.709 | INFO     | __main__:<cell line: 4>:4 - {
    "id": "7ded79fc-6347-487d-98e8-901aae59cd13",
    "name": "ep5g-vip-router",
    "tenant_id": "2379270948e848d99073172ae425701d",
    "admin_state_up": true,
    "status": "ACTIVE",
    "external_gateway_info": {
        "network_id": "717b5f2b-069e-4868-a24d-91a4ae3ad002",
        "external_fixed_ips": [
            {
                "subnet_id": "53d03ffd-0d49-4f9a-88a3-a30d69fe4827",
                "ip_address": "130.237.11.101"
            }
        ],
        "enable_snat": true
    },
    "description": "",
    "availability_zones": [],
    "availability_zone_hints": [],
    "routes": [],
    "flavor_id": null,
    "tags": [],
    "created_at": "2024-11-29T12:29:30Z",
    "updated_at": "2024-11-29T12:29:31Z",
    "revision_number": 3,
    "project_id": "2379270948e848d99073172ae425701d"
}
2024-11-29 12:29:36.46